In [4]:
from dotenv import load_dotenv
import os



In [5]:
load_dotenv()

host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [6]:
import pandas as pd
import folium 
import psycopg2
from folium.plugins import MarkerCluster
import requests
import json
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage
select_query = """
    SELECT *
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY senseid_fk ORDER BY time DESC) AS r
        FROM value
    ) sub
    WHERE r <= 4;
"""

# Abfrage ausführen
cursor.execute(select_query)

# Ergebnis abrufen
rows = cursor.fetchall()

# Ergebnis in ein Pandas DataFrame umwandeln
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Cursor und Verbindung schließen
cursor.close()
connection.close()


Verbindung zur Datenbank erfolgreich hergestellt.


In [7]:




# Load the data from the API
response = requests.get("https://us-central1-skillful-coast-423214-a6.cloudfunctions.net/lastEntries")
df_latest = pd.read_json(json.dumps(response.json()))

# Rename and clean up DataFrame columns
df_latest.columns = ["index", "time", "temperature", "luftfeuchtigkeit", "pm10", "pm2.5", "senseidfk", "r"]
df_latest.drop(columns="r", inplace=True)

# Connect to the PostgreSQL database to get station info
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Create a cursor to execute SQL queries
cursor = connection.cursor()

# SQL SELECT query to get stations
select_stations_query = "SELECT ort, lat, lon, senseid FROM box_id;"
cursor.execute(select_stations_query)
stations = cursor.fetchall()

# Initialize the map centered on Germany
map_deutschland = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

# Create a Marker Cluster
marker_cluster = MarkerCluster().add_to(map_deutschland)

# Add weather stations to the map
for station in stations:
    ort, lat, lon, senseid = station
    # Filter the DataFrame for the current station
    station_data = df_latest[df_latest['senseidfk'] == senseid]
    
    # Create a popup with the station's current data
    if not station_data.empty:
        latest_data = station_data.iloc[-1]
        popup_text = (
            f"Station: {ort}<br>"
            f"Time: {latest_data['time']}<br>"
            f"Temperature: {latest_data['temperature']} °C<br>"
            f"Luftfeuchtigkeit: {latest_data['luftfeuchtigkeit']} %<br>"
            f"PM10: {latest_data['pm10']} µg/m³<br>"
            f"PM2.5: {latest_data['pm2.5']} µg/m³"
        )
    else:
        popup_text = f"Station: {ort}<br>No data available."
    
    folium.Marker(
        location=[lat, lon],
        popup=popup_text
    ).add_to(marker_cluster)

# Display the map
map_deutschland.save('map_deutschland.html')



connection.close()

C:\Users\Benjamin\AppData\Local\Temp\ipykernel_3632\306686329.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_latest = pd.read_json(json.dumps(response.json()))


Verbindung zur Datenbank erfolgreich hergestellt.
